<a href="https://colab.research.google.com/github/luiz-oliveir/LSTM/blob/main/setup_and_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import subprocess
import shutil
from pathlib import Path
import time

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def run_command(cmd, check=True, shell=False):
    """Helper function to run commands and handle errors"""
    try:
        print(f"Executando comando: {' '.join(str(x) for x in cmd)}")
        result = subprocess.run(cmd, check=check, shell=shell,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
        print(f"Output: {result.stdout}")
        return result
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {' '.join(str(x) for x in cmd)}")
        print(f"Error output: {e.stderr}")
        return None

def clone_repository():
    """Clone the GitHub repository if in Colab"""
    if IN_COLAB:
        # Change to /content directory
        os.chdir('/content')

        # Remove existing directory if it exists
        if os.path.exists('/content/LSTM'):
            print("Removing existing repository...")
            shutil.rmtree('/content/LSTM')

        print("Cloning repository...")
        # Clone the repository
        cmd = ["git", "clone", "https://github.com/luiz-oliveir/LSTM.git"]
        result = run_command(cmd)

        if result is None or result.returncode != 0:
            print("Failed to clone repository")
            return False

        print("Repository cloned successfully")
        return True
    return True

def get_project_dir():
    """Get the project directory in Google Colab or local environment"""
    if IN_COLAB:
        project_dir = Path('/content/LSTM')
    else:
        project_dir = Path(os.getcwd()).parent

    print(f"Checking Colab path: {project_dir}")
    print(f"Path exists: {project_dir.exists()}")

    if not project_dir.exists():
        print("Project directory not found")
        return None

    print(f"Current directory: {project_dir}")
    print(f"Directory contents: {[f.name for f in project_dir.iterdir()]}")

    return project_dir

def setup_environment():
    """Setup the Python environment with required packages"""
    if not clone_repository():
        return False

    project_dir = get_project_dir()
    if not project_dir:
        print("Failed to locate project directory")
        return False

    if IN_COLAB:
        print("Installing requirements...")
        requirements_file = project_dir / 'requirements.txt'

        # Create requirements.txt if it doesn't exist
        if not requirements_file.exists():
            print("Creating requirements.txt...")
            requirements = """tensorflow>=2.8.0
numpy>=1.19.2
pandas>=1.3.0
scikit-learn>=0.24.2
keras-tuner>=1.1.0
matplotlib>=3.4.3"""
            with open(requirements_file, 'w') as f:
                f.write(requirements)
            print("Created requirements.txt")

        print("Installing dependencies...")
        cmd = [sys.executable, "-m", "pip", "install", "-r", str(requirements_file)]
        result = run_command(cmd)
        if result is None or result.returncode != 0:
            return False

    return True

def run_notebook():
    """Run the main notebook"""
    try:
        print("Executing LSTM VAE notebook...")
        cmd = ["jupyter", "nbconvert",
               "--to", "notebook",
               "--execute",
               "--inplace",
               "LSTM_VAE_com_ajustes.ipynb"]
        result = run_command(cmd)
        if result and result.returncode == 0:
            print("Successfully executed notebook")
            return True
        else:
            print("Error executing notebook")
            return False
    except Exception as e:
        print(f"Error running notebook: {e}")
        return False

if __name__ == '__main__':
    if setup_environment():
        project_dir = get_project_dir()
        if project_dir:
            os.chdir(project_dir)
            run_notebook()
        else:
            print("Failed to setup environment")